# 03 SDK | 08 Chat with Data

## Use Case

Businesses often face challenges when interacting with structured data, such as financial records or customer databases. A common scenario is the need for non-technical users to query and analyze this data without the expertise required to write complex SQL queries. For instance, an operations manager might want to quickly find out how many invoices exceeded a certain amount in a given year, but they are unfamiliar with SQL.

The use case revolves around enabling these users to "chat" with their data using natural language. By translating their questions into structured queries and executing them efficiently, businesses can make data more accessible and actionable, improving decision-making processes and reducing the reliance on technical teams.

## Solution Approach

This solution leverages Large Language Model (LLM) to translate user queries into SQL commands, which are then executed against a local database. The approach involves:

- **LLM Integration**: Using an LLM model to understand and convert the user's natural language query into a structured SQL query that the system can execute.
   
- **Client-Side Processing**: Executing the generated SQL queries on a lightweight, local database (e.g., SQLite) hosted on the client’s machine. This reduces server load and enhances the scalability of the solution.
   
- **Data Access and Query Execution**: The local database is populated with relevant data, ensuring that users can query and interact with the most up-to-date information. This data is isolated per user or tenant, ensuring security and customization.

By combining NLP with client-side compute, this solution enables efficient, scalable, and user-friendly data interaction, allowing businesses to offer robust data querying capabilities without the overhead of managing large-scale, server-based infrastructures.

## Prerequisites

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: System.Linq"
#r "nuget: Microsoft.Data.Sqlite, 8.0.6"
#r "nuget: System.Text.Json, 8.0.0"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.12 DotNetEnv, 2.5.0 Microsoft.Data.Sqlite, 8.0.6 System.Linq, 4.3.0 System.Text.Json, 8.0.0

### Instantiate OpenAI API

In [2]:
using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);


string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

Console.WriteLine($"OpenAI Client created...");

OpenAI Client created...


In [3]:
        string userQuery = "i need a list of all invoices in 2021";
        string jsonSchema = File.ReadAllText(Path.Combine(assetsFolder,"docs", "03_SDK", "sampleSchema.json"));
        var prompt = $@"
        You are an assistant that helps users translate their natural language queries into SQL queries for a specific database schema. Below is the schema of the database:

        Tables and Their Columns:

        1. financials
           - year (INTEGER)
           - month (INTEGER)
           - invoices (REAL)
           - payments (REAL)

        2. address
           - street (TEXT)
           - city (TEXT)
           - postalCode (TEXT)

        3. userInfo
           - name (TEXT)
           - email (TEXT)
           - taxId (TEXT)

        Please translate the following natural language queries into SQL queries and return the results in the following JSON format:

        ```json
        {{
          ""query"": ""<generated SQL query>"",
          ""error"": ""<error message if any>""
        }}
        ```

        Example Queries:
        1. ""Show me all the invoices for the year 2021.""
           ```json
           {{
             ""query"": ""SELECT * FROM financials WHERE year = 2021;"",
             ""error"": null
           }}
           ```

        2. ""How many months had invoices greater than 500?""
           ```json
           {{
             ""query"": ""SELECT COUNT(DISTINCT month) FROM financials WHERE invoices > 500;"",
             ""error"": null
           }}
           ```

        3. ""What is the email address of the user named John Doe?""
           ```json
           {{
             ""query"": ""SELECT email FROM userInfo WHERE name = 'John Doe';"",
             ""error"": null
           }}
           ```

        4. ""List all the postal codes.""
           ```json
           {{
             ""query"": ""SELECT postalCode FROM address;"",
             ""error"": null
           }}
           ```

        Now, translate the following query:

        Query: ""{userQuery}""
        Output:
        ";

In [18]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Diagnostics;

public class StructuredQueryResponse
{
    [JsonPropertyName("query")]
    public string Query { get; set; }

    [JsonPropertyName("error")]
    public string Error { get; set; }

    public string toString()
    {
        return $"Query: {Query}, Error: {Error}";
    }
}

public async Task<StructuredQueryResponse> GetStructuredQueryAsync(string userQuery)
{
    string sys_prompt = prompt;
    ChatCompletionsOptions simpleOption = new ChatCompletionsOptions()
    {
    //Request Properties
    ResponseFormat = ChatCompletionsResponseFormat.JsonObject,
    MaxTokens = 500,
    Temperature = 0.7f,
    NucleusSamplingFactor = 0.0f,
    FrequencyPenalty = 0.0f,
    PresencePenalty = 0.0f,
    DeploymentName = chatCompletionDeploymentName
    };

    simpleOption.Messages.Add(new ChatRequestSystemMessage(sys_prompt));
    var userMessage = $"Query: {userQuery}. Output:";
    simpleOption.Messages.Add(new ChatRequestUserMessage( userMessage));

    Response<ChatCompletions> simpleResponse = await openAIClient.GetChatCompletionsAsync(simpleOption);

    // Get the first choice from the response
    ChatCompletions simpleCompletions = simpleResponse.Value;

    string responseContent = simpleCompletions.Choices[0].Message.Content;

    // Deserialize the JSON response into a StructuredQueryResponse object
    StructuredQueryResponse structuredQueryResponse;
    try
    {
        structuredQueryResponse = JsonSerializer.Deserialize<StructuredQueryResponse>(responseContent);
    }
    catch (JsonException)
    {
        // Handle JSON parsing error
        structuredQueryResponse = new StructuredQueryResponse
        {
            Query = null,
            Error = "Failed to parse the response from the language model."
        };
    }

    return structuredQueryResponse;

}

In [19]:

var stopwatch = Stopwatch.StartNew();
// string jsonObject = File.ReadAllText(Path.Combine(assetsFolder,"docs", "03_SDK", "large_sample.json"));
string jsonObject = File.ReadAllText(Path.Combine(assetsFolder,"docs", "03_SDK", "large_sample.json"));
stopwatch.Stop();
Console.WriteLine($"Time taken to read the JSON file: {stopwatch.ElapsedMilliseconds} ms");

Time taken to read the JSON file: 36 ms


## Using sqllite


In [20]:

using System;
using Microsoft.Data.Sqlite;
using System.IO;
using System.Text.Json;
using System.Text.Json.Nodes;

In [21]:
public void SaveFinancialsTable(JsonNode jsonData, SqliteConnection connection)
{
    var financials = jsonData["financials"]["monthlyFinancials"].AsArray();
    using (var command = connection.CreateCommand())
    {
        command.CommandText = @"
        CREATE TABLE IF NOT EXISTS financials (
            year INTEGER,
            month INTEGER,
            invoices REAL,
            payments REAL
        )";
        command.ExecuteNonQuery();

        foreach (var item in financials)
        {
            command.CommandText = @"
            INSERT INTO financials (year, month, invoices, payments) 
            VALUES (@year, @month, @invoices, @payments)";
            command.Parameters.Clear();
            command.Parameters.AddWithValue("@year", item["year"].GetValue<int>());
            command.Parameters.AddWithValue("@month", item["month"].GetValue<int>());
            command.Parameters.AddWithValue("@invoices", item["invoices"].GetValue<decimal>());
            command.Parameters.AddWithValue("@payments", item["payments"].GetValue<decimal>());
            command.ExecuteNonQuery();
        }
    }
}

In [8]:
public void SaveAddressTable(JsonNode jsonData, SqliteConnection connection)
{
    var address = jsonData["address"];
    using (var command = connection.CreateCommand())
    {
        command.CommandText = @"
        CREATE TABLE IF NOT EXISTS address (
            street TEXT,
            city TEXT,
            postalCode TEXT
        )";
        command.ExecuteNonQuery();

        command.CommandText = @"
        INSERT INTO address (street, city, postalCode) 
        VALUES (@street, @city, @postalCode)";
        command.Parameters.Clear();
        command.Parameters.AddWithValue("@street", address["street"].GetValue<string>());
        command.Parameters.AddWithValue("@city", address["city"].GetValue<string>());
        command.Parameters.AddWithValue("@postalCode", address["postalCode"].GetValue<string>());
        command.ExecuteNonQuery();
    }
}


In [22]:
public void SaveUserInfoTable(JsonNode jsonData, SqliteConnection connection)
{
    var userInfo = jsonData["contactDetails"];
    var customerId = jsonData["customerId"].GetValue<string>();
    var taxId = jsonData["taxId"].GetValue<string>();
    var name = jsonData["name"].GetValue<string>();
    var currency = jsonData["currency"].GetValue<string>();
    var firstInvoiceDate = jsonData["firstInvoiceDate"].GetValue<string>();
    var foreignId = jsonData["foreignId"].GetValue<string>();

    using (var command = connection.CreateCommand())
    {
        command.CommandText = @"
        CREATE TABLE IF NOT EXISTS userInfo (
            customerId TEXT,
            phone TEXT,
            taxId TEXT,
            currency TEXT,
            firstInvoiceDate TEXT,
            foreignId TEXT,
            name TEXT
        )";
        command.ExecuteNonQuery();

        command.CommandText = @"
        INSERT INTO userInfo (customerId, phone, taxId, currency, firstInvoiceDate, foreignId, name) 
        VALUES (@customerId, @phone, @taxId, @currency, @firstInvoiceDate, @foreignId, @name)";
        command.Parameters.Clear();
        command.Parameters.AddWithValue("@customerId", customerId);
        command.Parameters.AddWithValue("@phone", userInfo["phone"].GetValue<string>());
        command.Parameters.AddWithValue("@taxId", taxId);
        command.Parameters.AddWithValue("@currency", currency);
        command.Parameters.AddWithValue("@firstInvoiceDate", firstInvoiceDate);
        command.Parameters.AddWithValue("@foreignId", foreignId);
        command.Parameters.AddWithValue("@name", name);
        command.ExecuteNonQuery();
    }
}


In [23]:
public void SaveJsonToSQLite(string jsonString)
{
    JsonNode jsonData = JsonNode.Parse(jsonString);

    string dbFilePath = Path.Combine(Directory.GetCurrentDirectory(), "data.db");

    using (var memoryConnection = new SqliteConnection("Data Source=:memory:"))
    {
        var stopwatch = Stopwatch.StartNew();
        memoryConnection.Open();
        SaveFinancialsTable(jsonData, memoryConnection);
        SaveAddressTable(jsonData, memoryConnection);
        SaveUserInfoTable(jsonData, memoryConnection);
        stopwatch.Stop();
        Console.WriteLine($"Time taken to create the in memory sqllite: {stopwatch.ElapsedMilliseconds} ms");

        // Backup the in-memory database to a file
        using (var fileConnection = new SqliteConnection($"Data Source={dbFilePath}"))
        {
            fileConnection.Open();
            memoryConnection.BackupDatabase(fileConnection);
        }

        memoryConnection.Close();
    }

    // Check if the file exists to confirm persistence
    if (File.Exists(dbFilePath))
    {
        Console.WriteLine($"Database successfully saved to {dbFilePath}");
    }
    else
    {
        Console.WriteLine($"Failed to save the database to {dbFilePath}");
    }
}

In [24]:

var stopwatch = Stopwatch.StartNew();

SaveJsonToSQLite(jsonObject);
stopwatch.Stop();
Console.WriteLine($"Time taken to create and persist the sqllite: {stopwatch.ElapsedMilliseconds} ms");



Time taken to create the in memory sqllite: 418 ms
Database successfully saved to /Users/yoavdobrin/workspace/fta/Azure-Samples/aoai-net-starterkit/docs/03_SDK/data.db
Time taken to create and persist the sqllite: 505 ms


In [25]:
public static void ExecuteSQLQuery(string dbFilePath, string sqlQuery)
{
    using (var connection = new SqliteConnection($"Data Source={dbFilePath}"))
    {
        connection.Open();
        using (var command = new SqliteCommand(sqlQuery, connection))
        {
            using (var reader = command.ExecuteReader())
            {
                while (reader.Read())
                {
                    for (int i = 0; i < reader.FieldCount; i++)
                    {
                        Console.Write($"{reader.GetName(i)}: {reader[i]} ");
                    }
                    Console.WriteLine();
                }
            }
        }
    }
}

In [26]:
string userQuery = "i wonder how many invoices were over 20000 in the year 21";
string dbFilePath = "data.db";
var stopwatch = Stopwatch.StartNew();
StructuredQueryResponse sqlQuery = await GetStructuredQueryAsync(userQuery);
stopwatch.Stop();
Console.WriteLine($"Time for SQL Query {stopwatch.ElapsedMilliseconds}: {sqlQuery.Query}");
stopwatch = Stopwatch.StartNew();
ExecuteSQLQuery(dbFilePath, sqlQuery.Query);
stopwatch.Stop();
Console.WriteLine($"Time taken to run the query: {stopwatch.ElapsedMilliseconds} ms");

Time for SQL Query 1484: SELECT COUNT(*) FROM financials WHERE invoices > 20000 AND year = 2021;
COUNT(*): 3252 
Time taken to run the query: 8 ms


## Conclusion

This notebook demonstrates a practical approach to enhancing data accessibility through the use of LLM and client-side compute. By translating user queries into SQL commands and executing them locally, we've shown how businesses can empower non-technical users to interact with structured data effectively.

The solution not only simplifies the querying process but also addresses key challenges such as: **server load, scalability, and cost efficiency**. By offloading the computational work to client devices and leveraging lightweight databases like SQLite, organizations can provide a responsive and energy-efficient data interaction experience.

This approach is particularly beneficial for ISVs and small to medium-sized businesses that need to offer robust data querying capabilities without the complexity and expense of maintaining large-scale server infrastructure. As businesses continue to seek more efficient and user-friendly ways to interact with their data, solutions like this will play a crucial role in bridging the gap between technical and non-technical users.

